### Notebook para povoar a PetroKGraph ontology com as entidades e relações extraídas do PetroNER teste

In [1]:
from owlready2 import *
import json

### Carregando listas com entidades e relações

In [2]:
# Carregando entidades linkadas
with open('Linked_entities', 'r') as f:
    Linked_entities = json.load(f)

# Carregando entidades novas
with open('New_entities', 'r') as f:
    New_entities = json.load(f)
    
# Carregando novas relações
with open('New_relations', 'r') as f:
    New_relations = json.load(f)

### Carregando PetroKGraph (ontologia povoada) e consultando a classe de cada entidade linkada

In [3]:
#Carregando PetroKGraph (ontologia povoada)
PetroKGraph_world = World()
PetroKGraph = PetroKGraph_world.get_ontology("../../../KnowledgeGraph/OntoGeoLogicaInstanciasRelacoes.owl").load()
# PetroKGraph.load()

In [4]:
# lista com as Classes das entidades linkadas
classes = []

for ent in Linked_entities:
    
    classe = PetroKGraph[ent[1:]].is_a
    classe_str = str(classe[0]).split('.')[1]
    classes.append(classe_str)

In [5]:
# Apagando da memória a ontologia povoada
# PetroKGraph.destroy()

### Carregando ontologia não povoada

In [6]:
#Carregando ontologias (não povoada)
onto = get_ontology("../../../KnowledgeGraph/OntoGeoLogicaEntidadesNomeadas.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [7]:
# Povoando a ontologia com as entidades linkadas

classes_w_instances = ['basin', 'field', 'geological_structure', 'well', 'textura', 'lithostratigraphic_unit']

classes_w_sub_classes = ['rock', 'unconsolidated_earth_material', 'o&g_earth_fluid', 'earth_fluid', 
                         'anthropogenic_fluid', 'petroleum_system_events', 'petroleum_system_elements']

crono = ['geological_age', 'geological_time_interval']

for n in range(len(Linked_entities)):
    # print (Linked_entities[n][1:], ' -- ', classes[n])
    # Verificando se a entidade linkada não é uma classe que já está na ontologia
    if Linked_entities[n][1:] not in classes_w_instances + classes_w_sub_classes + crono:
        
        # Povoando as classes com instâncias
        if classes[n] in classes_w_instances:
            with onto:
                onto[classes[n]](Linked_entities[n][1:])

        # Povoando as classes com subclasses
        if classes[n] in classes_w_sub_classes:
            with onto:
                types.new_class(Linked_entities[n][1:], (onto[classes[n]],))
                
        if classes[n] in crono:
            with onto:
                types.new_class(Linked_entities[n][1:] + '_quality', (onto['geological_age'],))
                onto['geological_time_interval'](Linked_entities[n][1:])

### Povoar entidades novas  
Como as URIs das entidades novas não possuem padronização, o script de entity link gera uma nova URI diferente da URI gerada no momento de anotação do PetroNER. Portanto, criamos um DExPARA de URIs para que seja realizada essa compatibilização. Também usamos esse DExPARA para ajustar URI's criadas automaticamente que não estão de acordo com a sintaxe do RDF.

In [8]:
de_para_novas_URIs = {'#NEW_BACIA_Bacia_de_Paris': '#INTER_BACIA_012',
                      '#NEW_BACIA_Bacia_de_o_Gabão': '#INTER_BACIA_017',
                      '#NEW_BACIA_Bacia_Lusitânica': '#INTER_BACIA_003',
                      '#NEW_UNIDADE_LITO_Formação_Vale_de_as_Fontes': '#inter_formacao_006',
                      '#NEW_UNIDADE_LITO_Membro_Mcnb': '#inter_membro_003',
                      '#NEW_UNIDADE_LITO_Formação_Cabo_Carvoeiro': '#inter_formacao_002',
                      '#NEW_UNIDADE_CRONO_Pliensbaquiano_–': '#NEW_UNIDADE_CRONO_Pliensbaquiano'} # URI que gerou erro no RDF
                      
                      # '#NEW_ROCHA_calcilutitos': '#calcilutite',
                      # '#NEW_UNIDADE_LITO_Formação_Lemede': '#inter_formacao_001',
                      # }
    
    
                    # '#NEW_BACIA_Tucano' : '#PetroNER_BACIA_Bacia_de_o_Tucano' ,
                    #  '#NEW_UNIDADE_CRONO_Neoaptiano' : '#PetroNER_UNIDADE_CRONO_Neoaptiano' ,
                    #  '#NEW_ROCHA_calcilutitos' : '#PetroNER_ROCHA_calcilutitos' ,
                    #  '#NEW_UNIDADE_LITO_Membro_Oiteirinhos' : '#PetroNER_UNIDADE_LITO_Membro_Oiteirinhos' ,
                    #  '#NEW_POÇO_poços_5_-_CPB_-_1A_-_SE' : '#PetroNER_POÇO_poços_5_-_CPB_-_1A_-_SE' ,
                    #  '#NEW_FLUIDO_fluidos' : '#PetroNER_FLUIDO_fluidos' ,
                    #  '#NEW_BACIA_Bacia_de_Paris' : '#PetroNER_BACIA_Bacia_de_Paris' ,
                    #  '#NEW_BACIA_bacia' : '#PetroNER_BACIA_bacia' ,
                    #  '#NEW_POÇO_Bu_-_91' : '#PetroNER_POÇO_Bu_-_91' ,
                    #  '#NEW_POÇO_14_-_PC_-_06' : '#PetroNER_POÇO_14_-_PC_-_06' ,
                    #  '#NEW_UNIDADE_CRONO_Sinemuriano' : '#PetroNER_UNIDADE_CRONO_Sinemuriano' ,
                    #  '#NEW_UNIDADE_LITO_Formação_Afligidos' : '#PetroNER_UNIDADE_LITO_Formação_Afligidos' ,
                    #  '#NEW_BACIA_Bacia_de_o_Gabão' : '#PetroNER_BACIA_Bacia_de_o_Gabão' ,
                    #  '#NEW_UNIDADE_LITO_Formação_Maracangalha' : '#PetroNER_UNIDADE_LITO_Formação_Maracangalha' ,
                    #  '#NEW_BACIA_bacia_de_Recife_-_João_Pessoa' : '#PetroNER_BACIA_bacia_de_Recife_-_João_Pessoa' ,
                    #  '#NEW_UNIDADE_LITO_Membro_Pedrão' : '#PetroNER_UNIDADE_LITO_Membro_Pedrão' ,
                    #  '#NEW_BACIA_Bacia_Lusitânica' : '#PetroNER_BACIA_Bacia_Lusitânica' ,
                    #  '#NEW_UNIDADE_CRONO_Neotriássico' : '#PetroNER_UNIDADE_CRONO_Neotriássico' ,
                    #  '#NEW_UNIDADE_CRONO_Toarciano_inferior' : '#PetroNER_UNIDADE_CRONO_Toarciano_inferior' ,
                    #  '#NEW_UNIDADE_LITO_Formação_Vale_de_as_Fontes' : '#PetroNER_UNIDADE_LITO_Formação_Vale_de_as_Fontes' ,
                    #  '#NEW_UNIDADE_LITO_Formação_Lemede' : '#PetroNER_UNIDADE_LITO_Formação_Lemede' ,
                    #  '#NEW_UNIDADE_LITO_Membro_Mcnb' : '#PetroNER_UNIDADE_LITO_Membro_Mcnb' ,
                    #  '#NEW_POÇO_poço_2_-_SBST_-_1_-_BA' : '#PetroNER_POÇO_poço_2_-_SBST_-_1_-_BA' ,
                    #  '#NEW_ROCHA_dinocistos' : '#PetroNER_ROCHA_dinocistos'}


In [9]:
de_para_classes = {  'BACIA' :  'basin',
                     'CAMPO' :  'field',
                     'ESTRUTURA_FÍSICA' :  'geological_structure',
                     'EVENTO_PETRO' :  'petroleum_system_events',
                     'FLUIDO' :  'anthropogenic_fluid',
                     'FLUIDODATERRA_i' :  'o&g_earth_fluid',
                     'FLUIDODATERRA_o' :  'earth_fluid',
                     'NÃOCONSOLID' :  'unconsolidated_earth_material',
                     'POÇO' :  'well',
                     'ROCHA' :  'rock',
                     'TEXTURA' :  'textura',
                     'UNIDADE_CRONO' :  'geological_age',
                     'UNIDADE_LITO' :  'lithostratigraphic_unit',
                     'ELEMENTO_PETRO' :  'petroleum_system_elements'}


In [10]:
# Iterando por cada entidade nova
for key in New_entities.keys():
    
    #Verificando se a URI está no DE x PARA de URIs
    try:
        uri = de_para_novas_URIs[New_entities[key]['URI']][1:]
    except:
        uri = New_entities[key]['URI'][1:]
                                 
    
    classe_n = de_para_classes[New_entities[key]['classe']]
    
    # Povoando as classes com instâncias
    if classe_n in classes_w_instances:
        with onto:
            onto[classe_n](uri)

    # Povoando as classes com subclasses
    if classe_n in classes_w_sub_classes:
        with onto:
            types.new_class(uri, (onto[classe_n],))

    if classe_n in crono:
        with onto:
            types.new_class(uri + '_quality', (onto['geological_age'],))
            onto['geological_time_interval'](uri)


### Povoar relações

In [11]:
# Verificar se a URI presente na lista de relações tem que ser substituida por outra URI da lista DE x PARA.

new_list = []
# Iterando por todas as relações
for rel in New_relations:
    ent_1 = None
    ent_2 = None
    #checando lista DExPARA da entidade 1
    try:
        ent_1 = de_para_novas_URIs[rel[0]][1:]
    except:
        ent_1 = rel[0][1:]
        
    try:
        ent_2 = de_para_novas_URIs[rel[2]][1:]
    except:
        ent_2 = rel[2][1:]
    
    # Algumas relações preditas podem não ser permitida na ontologia, o que acarreta erro. 
    # Nesse caso, a relação não será incluida no grafo. (Incluir alguma checagem no script de predição de relações)
    try:
        if rel[1] == 'constituted_by':
            onto[ent_1].constituted_by.append(onto[ent_2])

        if rel[1] == 'crosses':
            onto[ent_1].crosses.append(onto[ent_2])

        if rel[1] == 'has_age':
            # Criando a instância "idade geológica da unidade lito"
            onto[ent_2 + '_quality'](ent_2 + '_quality_' + ent_1)

            # Criando a relação entra a unidade lito e instância "idade geológica da unidade lito"
            onto[ent_1].has_age.append(onto[ent_2 + '_quality_' + ent_1])

            # Adicionando a relação partipates_in entre a qualidade Geological age e o Geological Time interval

            onto[ent_2 + '_quality_' + ent_1].participates_in.append(onto[ent_2])

        if rel[1] == 'participates_in':
             onto[ent_1].participates_in.append(onto[ent_2])
                
        if rel[1] == 'located_in':
             onto[ent_1].located_in.append(onto[ent_2])

        if rel[1] == 'part_of':
             onto[ent_1].part_of.append(onto[ent_2])

        if rel[1] == 'carrier_of':
             onto[ent_1].carrier_of.append(onto[ent_2])

        if rel[1] == 'generated_by':
             onto[ent_1].generated_by.append(onto[ent_2])

        if rel[1] == 'has_temporal_relation':
            onto[ent_1].has_temporal_relation.append(onto[ent_2])
    except:
        pass

### Salvar grafo

In [12]:
onto.save(file = 'PetroKGraph_pred')